In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MeasureControl
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, MultiPoint, MultiLineString
from shapely.ops import split, snap, substring, nearest_points
from shapely.affinity import scale, rotate
import matplotlib.pyplot as plt
import math

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

module_path = os.path.abspath(os.path.join(r'..\BTools\beratools\tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from BTools.beratools.tools.line_footprint_fixed import *


from label_centerlines import get_centerline

lcpath = gpd.read_file(r"C:\BERATools\Surmont_New_AOI\test_selected_lines\centerlines_least_cost_path.shp")
cl = gpd.read_file(r"C:\BERATools\Surmont_New_AOI\test_selected_lines\centerlines.shp")
poly = gpd.read_file(r"C:\BERATools\Surmont_New_AOI\test_selected_lines\centerlines_corridor_poly.shp")

mapa = folium.Map([56.224917, -110.713430],
    zoom_start=17,
    tiles='cartodbpositron',
    max_zoom=24)

geoms = gpd.GeoDataFrame(geometry=[lcpath.geometry.iloc[0], cl.geometry.iloc[0], poly.geometry.iloc[0]], crs='EPSG:26912')
geoms['color'] = ['#E0DB4D', '#288519', '#553a70']
gjson = geoms.to_crs(epsg='4326').to_json()
# geoms.plot(color=geoms['color'])

folium.GeoJson(
    gjson,
    style_function=lambda feature: {
        "fillColor": feature["properties"]["color"],
        "color": feature["properties"]["color"],
        "weight": 2,
        "dashArray": "0, 0",
    },
).add_to(mapa)

In [ ]:
lcp = lcpath.geometry.iloc[0]
line_1 = substring(lcp, start_dist=0.0, end_dist=lcp.length/2)
line_2 = substring(lcp, start_dist=lcp.length/2, end_dist=lcp.length)
list(line_1.coords)[-1], list(line_2.coords)[0]

In [ ]:
pts =  shapely.force_2d([Point(list(lcp.coords)[0]), Point(list(line_1.coords)[-1]), Point(list(lcp.coords)[-1])])
pts

In [ ]:
#pt_perp = Point(list(line_1.coords)[-1])
perp = generate_perpendicular_line_precise(pts)
perp

In [ ]:
poly_exterior = Polygon(poly.geometry.iloc[0].buffer(1e-3).exterior)
poly_split = split(poly_exterior, perp)
poly_1 = poly_split.geoms[0]
poly_2 = poly_split.geoms[1]
poly_1

In [ ]:
poly_2

In [ ]:
print(poly_2.intersection(line_1).length, line_1.length)

In [ ]:
center_line_1 = find_centerline(poly_2, line_1)
center_line_1

In [ ]:
center_line_2 = find_centerline(poly_1, line_2)
center_line_2

In [ ]:
center_line_1.coords[0], center_line_1.coords[-1], line_1.coords[0], line_1.coords[-1]

In [ ]:
list(lcp.coords)[0], list(lcp.coords)[-1]

In [ ]:
geoms_cl = gpd.GeoDataFrame(geometry=[center_line_1, center_line_2], crs='EPSG:26912')
geoms_cl['color'] = ['#5E31DE', '#E70C4F']
gjson_cl = geoms_cl.to_crs(epsg='4326').to_json()

folium.GeoJson(
    gjson_cl,
    style_function=lambda feature: {
        "fillColor": feature["properties"]["color"],
        "color": feature["properties"]["color"],
        "weight": 2,
        "dashArray": "0, 0",
    },
).add_to(mapa)

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs(26912, 4326, always_xy=True)
center = shapely.force_2d(transform(transformer.transform, Point(list(line_1.coords)[-1])))
print(center)

In [ ]:
folium.Marker(location=[center.y, center.x], popup='center').add_to(mapa)

In [ ]:
mapa.add_child(MeasureControl())
mapa

In [ ]:
mapa.save(r'D:\Temp\test.html')